In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, month, min, max, avg
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline

Create Spark Session

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") 
    .config('spark.executor.memoryOverhead', '3000')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 17:33:55 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/19 17:33:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 17:33:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 17:34:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Loading Taxi Data (no shooting)

In [3]:
# Define a list of 3-month segments
segments = [
    "2016_01_to_03", "2016_04_to_06", "2016_07_to_09", "2016_10_to_12",
    "2017_01_to_03", "2017_04_to_06", "2017_07_to_09", "2017_10_to_12",
    "2018_01_to_03", "2018_04_to_06", "2018_07_to_09", "2018_10_to_12",
    "2019_01_to_03", "2019_04_to_06", "2019_07_to_09", "2019_10_to_12"
]

# Create a list to store the DataFrames
taxi_periods = []

# Create DataFrames for each segment and add to the list
for segment in segments:
    segment_df = spark.read.parquet(f'../data/raw/yellow_taxi_hourly/{segment}.parquet/*')
    taxi_periods.append(segment_df)
    

Loading Shooting Trip Data

In [4]:
# Define a list of 3-month segments
segments = [
    "2016_01_to_03", "2016_04_to_06", "2016_07_to_09", "2016_10_to_12",
    "2017_01_to_03", "2017_04_to_06", "2017_07_to_09", "2017_10_to_12",
    "2018_01_to_03", "2018_04_to_06", "2018_07_to_09", "2018_10_to_12",
    "2019_01_to_03", "2019_04_to_06", "2019_07_to_09", "2019_10_to_12"
]

# Create a list to store the DataFrames
shooting_trips = []

# Create DataFrames for each segment and add to the list
for segment in segments:
    segment_df = spark.read.parquet(f'../data/curated/shooting_taxi_hour_data_reduced/{segment}.parquet/*')
    shooting_trips.append(segment_df)
    

Create Month column for shooting data and rename column as Hour (of pickup) - wasn't able to save other dataframes

In [5]:
# List to store modified DataFrames with the "month" and "hour" columns
timed_shooting_periods = []

# Loop through the list of DataFrames and add the "month" and "hour" columns
for taxi_df in shooting_trips:
    full_modified_taxi_df = taxi_df.withColumn("Month", month(col("Date"))).withColumnRenamed("pickup_hour", "Hour")
    timed_shooting_periods.append(full_modified_taxi_df)

Rename PULocationID as LocationID

In [6]:
# List to store modified DataFrames with the "month" and "hour" columns
timed_periods = []

# Loop through the list of DataFrames and add the "month" and "hour" columns
for taxi_df in taxi_periods:
    full_modified_taxi_df = taxi_df.withColumnRenamed("PULocationID", "LocationID")
    timed_periods.append(full_modified_taxi_df)

Join into one big dataframe - First Taxi, then shooting trips

In [7]:
# Combine all DataFrames in the taxi_periods list using union
combined_taxi_dataframe = reduce(DataFrame.unionAll, timed_periods)
combined_shooting_dataframe = reduce(DataFrame.unionAll, timed_shooting_periods)

Function to count size of dataframes

In [ ]:
def calculate_total_rows(dataframe):
    """
    Calculates the total number of rows in DataFrame and prints the result
    Inputs:
        DataFrame for which to calculate the total rows
    """
    total_rows = dataframe.count()
    print("Total rows:", total_rows)

Count size of shooting aggregated dataframe

In [ ]:
calculate_total_rows(combined_shooting_dataframe)

Count size of taxi aggregated dataframe

In [ ]:
calculate_total_rows(combined_taxi_dataframe)

Outlier Detection - Remove Outliers from number of trips

In [9]:
# Calculate the minimum and maximum values for the 'number_of_trips' column for shooting trips
min_value = combined_shooting_dataframe.select(min("number_of_trips")).first()[0]
max_value = combined_shooting_dataframe.select(max("number_of_trips")).first()[0]

# Print or display the range of values
print(f"Range of number_of_trips values (shooting): [{min_value}, {max_value}]")

Range of number_of_trips values (shooting): [1, 3111]


In [10]:
# Calculate the minimum and maximum values for the 'number_of_trips' column for taxi trips
min_value = combined_taxi_dataframe.select(min("number_of_trips")).first()[0]
max_value = combined_taxi_dataframe.select(max("number_of_trips")).first()[0]

# Print or display the range of values
print(f"Range of number_of_trips values (taxi): [{min_value}, {max_value}]")

Range of number_of_trips values (taxi): [1, 3746]


In [11]:
# Calculate mean and standard deviation
mean_std_df = combined_shooting_dataframe.selectExpr("avg(number_of_trips) as mean", "stddev(number_of_trips) as std_dev")
mean_std = mean_std_df.first()

# Calculate Z-score
z_score_df = combined_shooting_dataframe.withColumn("z_score", (col("number_of_trips") - mean_std.mean) / mean_std.std_dev)

# Set the Z-score threshold for removing outliers
z_score_threshold = 3.0

# Filter out rows with Z-scores above the threshold
remove_outliers_shooting_dataframe = z_score_df.filter(col("z_score") <= z_score_threshold)

In [12]:
# Calculate mean and standard deviation
mean_std_df = combined_taxi_dataframe.selectExpr("avg(number_of_trips) as mean", "stddev(number_of_trips) as std_dev")
mean_std = mean_std_df.first()

# Calculate Z-score
z_score_df = combined_taxi_dataframe.withColumn("z_score", (col("number_of_trips") - mean_std.mean) / mean_std.std_dev)

# Set the Z-score threshold for removing outliers
z_score_threshold = 3.0

# Filter out rows with Z-scores above the threshold
remove_outliers_taxi_dataframe = z_score_df.filter(col("z_score") <= z_score_threshold)

In [13]:
# Calculate the minimum and maximum values for the 'number_of_trips' column
min_value = remove_outliers_shooting_dataframe.select(min("number_of_trips")).first()[0]
max_value = remove_outliers_shooting_dataframe.select(max("number_of_trips")).first()[0]

# Print or display the range of values
print(f"Range of number_of_trips values (shooting): [{min_value}, {max_value}]")

Range of number_of_trips values (shooting): [1, 175]


In [14]:
# Calculate the minimum and maximum values for the 'number_of_trips' column
min_value = remove_outliers_taxi_dataframe.select(min("number_of_trips")).first()[0]
max_value = remove_outliers_taxi_dataframe.select(max("number_of_trips")).first()[0]

# Print or display the range of values
print(f"Range of number_of_trips values (taxi): [{min_value}, {max_value}]")

Range of number_of_trips values (taxi): [1, 610]


Size of shooting data after outliers

In [ ]:
calculate_total_rows(remove_outliers_shooting_dataframe)

Size of taxi data after outliers

In [ ]:
calculate_total_rows(remove_outliers_taxi_dataframe)

Rename the variables 

In [15]:
# rename dataframes
combined_shooting_dataframe = remove_outliers_shooting_dataframe
combined_taxi_dataframe = remove_outliers_taxi_dataframe

One Hot Encoding to later be combined

First Taxi Data

In [17]:
# Specify the columns to one-hot encode
columns_to_encode = ["LocationID", "Month", "Hour"]

# Create a list to store the encoded columns
encoded_columns = []

for column in columns_to_encode:
    # Apply OneHotEncoder to the specified column
    encoder = OneHotEncoder(inputCol=column, outputCol=f"{column}_encoded")
    
    # Create a pipeline to apply the transformation
    pipeline = Pipeline(stages=[encoder])
    combined_taxi_dataframe = pipeline.fit(combined_taxi_dataframe).transform(combined_taxi_dataframe)
    
    # Add the encoded column to the list
    encoded_columns.append(f"{column}_encoded")

# Select the original columns and the encoded columns
selected_columns = ["LocationID", "Date", "Weekend", "number_of_trips"] + encoded_columns
combined__taxi_dataframe = combined_taxi_dataframe.select(*selected_columns)


Now for shooting trips

In [18]:
# Specify the columns to one-hot encode
columns_to_encode_shooting = ["LocationID", "Month", "Hour"]

# Create a list to store the encoded columns
encoded_columns_shooting = []

for column in columns_to_encode_shooting:
    # Apply OneHotEncoder to the specified column
    encoder = OneHotEncoder(inputCol=column, outputCol=f"{column}_encoded")
    
    # Create a pipeline to apply the transformation
    pipeline = Pipeline(stages=[encoder])
    combined_shooting_dataframe = pipeline.fit(combined_shooting_dataframe).transform(combined_shooting_dataframe)
    
    # Add the encoded column to the list
    encoded_columns_shooting.append(f"{column}_encoded")

# Select the original columns and the encoded columns
selected_columns_shooting = ["LocationID", "Date", "Weekend", "number_of_trips"] + encoded_columns_shooting
combined_shooting_dataframe = combined_shooting_dataframe.select(*selected_columns_shooting)

Now to save onehot encoded data

In [20]:
# Save the DataFrame as a Parquet file
output_path = "../data/curated/onehot_shooting_data.parquet"
combined_shooting_dataframe.write.parquet(output_path, mode="overwrite")

print("DataFrame saved as Parquet file.")

DataFrame saved as Parquet file.


In [21]:
# Save the DataFrame as a Parquet file
output_path = "../data/curated/onehot_taxi_data.parquet"
combined_taxi_dataframe.write.parquet(output_path, mode="overwrite")

print("DataFrame saved as Parquet file.")


# Stop the SparkSession
spark.stop()

DataFrame saved as Parquet file.
